In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [ ]:
import os
folder = "/Users/jacquelinezhu/Desktop/SleepdB/acoustic_project/04162025_05142025"
input = os.path.join(folder, "AcousticFeatures_April16_May14_soo.xlsx")
#df = pd.read_excel(input).dropna(how='all')

In [ ]:
def preprocess(original: pd.Series) -> tuple[pd.Series, pd.Series]:
    
    original = pd.to_numeric(original, errors='coerce')
    log = np.log1p(original)()

    #transformer = PowerTransformer(method='yeo-johnson', standardize=False)
    #yeo_arr = transformer.fit_transform(original.values.reshape(-1, 1))
    #yeo = pd.Series(yeo_arr.ravel(), index=original.index)

    return original, log

In [ ]:
def plot(series: pd.Series):
    feature_name = "AHI"
    original, log_transformed = preprocess(series)

    n_samples = len(original)
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    sns.histplot(original, kde=True, bins=n_samples, ax=axes[0], color="steelblue")
    axes[0].set_title(f"Original {feature_name}")

    sns.histplot(log_transformed, kde=True, bins=n_samples, ax=axes[1], color="orange")
    axes[1].set_title(f"Log {feature_name}")

    plt.tight_layout()
    plt.show()

plot(df['AHI'])

In [ ]:
def plot2(df: pd.DataFrame):
    for i in range(0, 7, 2):
        col1 = f"spec_contrast{i}"
        col2 = f"spec_contrast{i+1}"

        if col1 in df.columns and col2 in df.columns:
            fig, axes = plt.subplots(1, 2, figsize=(16, 5))  # Two subplots

            series1 = pd.to_numeric(df[col1], errors='coerce')
            n_samples1 = len(series1)
            sns.histplot(series1, kde=True, bins=n_samples1, color="steelblue", ax=axes[0])
            axes[0].set_title(f"Original {col1}")

            series2 = pd.to_numeric(df[col2], errors='coerce')
            n_samples2 = len(series2)
            sns.histplot(series2, kde=True, bins=n_samples2, color="steelblue", ax=axes[1])
            axes[1].set_title(f"Original {col2}")

            plt.tight_layout()
            plt.show()

plot2(df)

In [29]:
import os

folder = "/Users/jacquelinezhu/Desktop/SleepdB/acoustic_project/05142025"

input_file = os.path.join(folder, "AcousticFeatures_May14_AHI.xlsx")
output_file = os.path.join(folder, "AcousticFeatures_May14_AHI_2_scaled2.xlsx")


df = pd.read_excel(input_file).dropna(how='all')
df = df.apply(pd.to_numeric, errors='coerce')

scaled_features = {}

for feature_name in df.columns:
    original_col = df[feature_name]
    if original_col.dropna().empty:
        continue
    try:
        _, _, scaled = preprocess(original_col)
    except Exception as e:
        print(f"⚠️ Skipping {feature_name} due to: {e}")
        scaled = original_col.copy() 
    
    scaled_full = scaled.reindex(df.index)
    scaled_features[feature_name] = scaled_full

scaled_df = pd.DataFrame(scaled_features)
scaled_df.to_excel(output_file, index=False)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:3256: RuntimeWarning: overflow encountered in power
  out[pos] = (np.power(x[pos] + 1, lmbda) - 1) / lmbda


⚠️ Skipping bandPow0_10 due to: The algorithm terminated without finding a valid bracket. Consider trying different initial points.


In [ ]:
def plot(series: pd.Series, log_series: pd.Series, scaled_series: pd.Series, feature_name: str) -> None:
    data_list = [series, log_series, scaled_series]
    titles = ["Original", "Yeo Johnson", "Scaled"]

    plt.figure(figsize=(12, 3))
    for i, (data, title) in enumerate(zip(data_list, titles), start=1):
        plt.subplot(1, 3, i)
        sns.histplot(data, bins=37, kde=True)
        plt.title(f"{title}: {feature_name}")

    plt.tight_layout()
    plt.show()

In [ ]:
# bandpower

folder = "/Users/jacquelinezhu/Desktop/SleepdB/acoustic_project/05142025"
input_file = os.path.join(folder, "AcousticFeatures_May14_AHI_2.xlsx")
output_file = os.path.join(folder, "AcousticFeatures_May14_AHI_2_scaled_bandpower.xlsx")

feature_name = "bandPow0_10"

df = pd.read_excel(input_file).dropna(how='all')
df = df.apply(pd.to_numeric, errors='coerce')

feature = df[feature_name].replace([np.inf, -np.inf], np.nan).dropna()

scaler = StandardScaler()
scaled = pd.Series(
    scaler.fit_transform(feature.values.reshape(-1, 1)).ravel(),
    index=feature.index
)

scaled_df = pd.DataFrame({feature_name: scaled.reindex(df.index)})
scaled_df.to_excel(output_file, index=False)